# Import librairies

In [19]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Import file(s)

In [2]:
df = pd.read_csv("lexicon_score_per_user_df.csv").drop(columns=["Unnamed: 0", "user"])

In [3]:
df.head()

,Politique,Actualité,Candidats,Travail,Santé,Economie,Sécurité,Extrême,Ecologie,Pouvoir_achat,name,political_orientation
0,166,204,90,63,63,18,63,5,11,3,Alex Langlois,divers
1,160,117,60,15,24,22,15,4,2,1,Fabrice Grimal,divers
2,125,125,59,51,80,52,49,18,12,2,Edouard Philippe,droite
3,38,51,8,13,13,4,12,3,3,0,Clara Egger,divers
4,174,175,57,44,24,51,45,28,17,4,Georges Kuzmanovic,divers


# KNN

In [98]:
def predict_nearest_neighbour(name):
    # Define the main df
    df_copy = df.copy()
    
    # Store our candidate's index in the DataFrame
    index_candidate = df_copy[df_copy["name"]==name].index[0]

    # Encode the target and create a feature "name_encoded"
    le = LabelEncoder()
    le.fit(df_copy["name"])
    df_copy["encode_name"] = le.transform(df_copy["name"])

    # Retrieve political_orientation and name
    df_copy = df_copy.drop(columns=["name", "political_orientation"])

    # Define X and y
    X = df_copy.drop(columns=["encode_name"])
    y = df_copy['encode_name']

    # Scale X
    scaler = MinMaxScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)

    # Retrieve the line for the candidate we are working on with his/her index in the main df
    X_scaled_train = pd.DataFrame(X_scaled).drop([index_candidate]).to_numpy()
    y_train = pd.DataFrame(y).drop([index_candidate]).to_numpy().ravel()

    X_scaled_test = pd.DataFrame(X_scaled).iloc[index_candidate].to_numpy().reshape(1, -1)
    # y_test = pd.DataFrame(y).iloc[index_candidate]

    # Define the model and fit it with only one neighbour because we want to predict the nearest one
    model = KNeighborsClassifier(n_neighbors=1)
    # Fit on X_scaled_train and y_train
    model.fit(X_scaled_train, y_train)

    # Predict 
    y_pred = model.predict(X_scaled_test)
    predictions = le.inverse_transform(y_pred)

    return predictions[0]

In [99]:
for candidate in df["name"].sort_values():
    print(f"{candidate} : nearest_neighbour is {predict_nearest_neighbour(name=candidate)}")

Alex Langlois : nearest_neighbour is Anne Hidalgo
Anasse Kazib : nearest_neighbour is Eric Zemmour
Anna Agueb-Porterie : nearest_neighbour is Charlotte Marchandise
Anne Hidalgo : nearest_neighbour is Denis Payre
Antoine Martinez : nearest_neighbour is Georges Kuzmanovic
Antoine Waechter : nearest_neighbour is Charlotte Marchandise
Arnaud Montebourg : nearest_neighbour is Nathalie Arthaud
Charlotte Marchandise : nearest_neighbour is Anna Agueb-Porterie
Clara Egger : nearest_neighbour is Gildas Vieira
Denis Payre : nearest_neighbour is Anna Agueb-Porterie
Edouard Philippe : nearest_neighbour is Antoine Martinez
Emmanuel Macron : nearest_neighbour is Pierre Larrouturou
Eric Ciotti : nearest_neighbour is Philippe Juvin
Eric Zemmour : nearest_neighbour is Georges Kuzmanovic
Fabien Roussel : nearest_neighbour is Nathalie Arthaud
Fabrice Grimal : nearest_neighbour is Luc Laforets
Florian Philippot : nearest_neighbour is Jean-Frédéric Poisson
François Asselineau : nearest_neighbour is Nicolas 